In [3]:
#install libraries
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00 0:00:01


In [4]:
#include libraries
import os
from PyPDF2 import PdfReader, PdfWriter

In [106]:
#receives pdf through the input path
#return the first page number on which the phrase exists
#if phrase is not found: returns -1
def find_transcript_starting_page(input_pdf_path, phrase):
    with open(input_pdf_path, 'rb') as infile:
        reader = PdfReader(infile)

        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()

            if phrase in text:
                return page_num

        return -1

In [111]:
#receives pdf through the input path
#writes modified pdf to output path
#modified pdf starts at starting page and cuts out the top and bottom of the page
def remove_annotations(input_pdf_path, output_pdf_path, starting_page):
    with open(input_pdf_path, 'rb') as infile:
        reader = PdfReader(infile)
        writer = PdfWriter()

        for page_num in range(starting_page, len(reader.pages)):
            page = reader.pages[page_num]
            page.mediabox.lower_left = (page.mediabox.left, page.mediabox.bottom + 55)
            page.mediabox.upper_right = (page.mediabox.right, page.mediabox.top - 60)  # Adjust this value as per your requirement to crop footnotes

            writer.add_page(page)

        with open(output_pdf_path, 'wb') as outfile:
            writer.write(outfile)

In [109]:
# Path to the directory containing your PDFs
pdf_directory = "/content/in"

# Output directory to store modified PDFs
output_directory = "/content/out"

# phrase that marks the starting page of the pages that should be included
phrase = "FINAL TRANSCRIPT"

# Iterate through each PDF in the directory
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        input_pdf_path = os.path.join(pdf_directory, filename)
        output_pdf_path = os.path.join(output_directory, filename)

        #get starting page
        transcript_starting_page = find_transcript_starting_page(input_pdf_path, phrase)

        #if the phrase was found, the document will be edited and written to the out folder
        if transcript_starting_page > 0: remove_annotations(input_pdf_path, output_pdf_path, transcript_starting_page)